In [5]:
import pandas as pd

# Load the dataset
df = pd.read_excel("Online Retail.xlsx")

# Show basic info
print("Shape of dataset:", df.shape)
print("\nFirst 5 rows:\n", df.head())
print("\nSummary:\n", df.describe(include='all'))
print("\nMissing values:\n", df.isnull().sum())

Shape of dataset: (541909, 8)

First 5 rows:
   InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

          InvoiceDate  UnitPrice  CustomerID         Country  
0 2010-12-01 08:26:00       2.55     17850.0  United Kingdom  
1 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
2 2010-12-01 08:26:00       2.75     17850.0  United Kingdom  
3 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  
4 2010-12-01 08:26:00       3.39     17850.0  United Kingdom  

Summary:
         InvoiceNo StockCode                         Description  \
count    541909.0    541909                              540455  

In [6]:
import pandas as pd

# Load data
df = pd.read_excel("Online Retail.xlsx")

# Drop rows with missing CustomerID
df.dropna(subset=['CustomerID'], inplace=True)

# Remove canceled invoices (InvoiceNo starts with 'C')
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]

# Keep only positive Quantity
df = df[df['Quantity'] > 0]

# Show cleaned shape
print("Cleaned data shape:", df.shape)

# Show top 10 countries by transaction count
top_countries = df['Country'].value_counts().head(10)
print("\nTop 10 countries by number of transactions:\n", top_countries)

# Save top 3 countries for next step
top_3_countries = top_countries.head(3).index.tolist()
print("\nTop 3 countries for basket analysis:", top_3_countries)


Cleaned data shape: (397924, 8)

Top 10 countries by number of transactions:
 Country
United Kingdom    354345
Germany             9042
France              8342
EIRE                7238
Spain               2485
Netherlands         2363
Belgium             2031
Switzerland         1842
Portugal            1462
Australia           1185
Name: count, dtype: int64

Top 3 countries for basket analysis: ['United Kingdom', 'Germany', 'France']


In [7]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# Load and clean data again
df = pd.read_excel("Online Retail.xlsx")
df.dropna(subset=['CustomerID'], inplace=True)
df = df[~df['InvoiceNo'].astype(str).str.startswith('C')]
df = df[df['Quantity'] > 0]

# Select top 3 countries
top_countries = df['Country'].value_counts().head(3).index.tolist()

# Loop through each country
for country in top_countries:
    print(f"\n🛒 Analyzing country: {country}")
    
    # Filter data for country
    country_data = df[df['Country'] == country]
    
    # Create basket (InvoiceNo x Description)
    basket = (
        country_data
        .groupby(['InvoiceNo', 'Description'])['Quantity']
        .sum()
        .unstack()
        .fillna(0)
    )
 
    # Convert to boolean: True if quantity > 0, else False
    basket = basket > 0

    # Apply Apriori
    frequent_itemsets = apriori(basket, min_support=0.02, use_colnames=True)
    print("Frequent itemsets found:", len(frequent_itemsets))

    # Generate rules
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
    rules = rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
    
    # Display top 5 rules
    print("\nTop 5 rules:")
    print(rules.head())



🛒 Analyzing country: United Kingdom
Frequent itemsets found: 235

Top 5 rules:
                            antecedents                           consequents  \
0          (ALARM CLOCK BAKELIKE GREEN)           (ALARM CLOCK BAKELIKE RED )   
1           (ALARM CLOCK BAKELIKE RED )          (ALARM CLOCK BAKELIKE GREEN)   
2  (GARDENERS KNEELING PAD CUP OF TEA )   (GARDENERS KNEELING PAD KEEP CALM )   
3   (GARDENERS KNEELING PAD KEEP CALM )  (GARDENERS KNEELING PAD CUP OF TEA )   
4     (GREEN REGENCY TEACUP AND SAUCER)      (PINK REGENCY TEACUP AND SAUCER)   

    support  confidence       lift  
0  0.027269    0.657971  14.451925  
1  0.027269    0.598945  14.451925  
2  0.027509    0.730463  16.390122  
3  0.027509    0.617251  16.390122  
4  0.024266    0.660131  22.293137  

🛒 Analyzing country: Germany
Frequent itemsets found: 915

Top 5 rules:
                            antecedents  \
0                             (POSTAGE)   
1              (10 COLOUR SPACEBOY PEN)   
2        